# Google Code Jam 
## Kickstart Round H 2018

By Winn Chow, winnchow@gmail.com

C. Let Me Count The Ways

https://code.google.com/codejam/contest/3324486/dashboard#s=p2

### Prepare Test Cases - Solve A-small

In [ ]:
filename_s = "C-small-practice.in"

In [21]:
def read_data_file(filename):
    """Read input file and return test cases in a list."""
    
    with open(filename, "r") as f:
        # Read number of test cases
        n = int(f.readline())
        print("n = " + str(n))
        # Read all the rest
        input_cases = f.readlines()

    # Read each test case
    def read_case(j):
        tmp = input_cases[j].split()
        # total number of numbers
        n_test = int(tmp[0])
        m_test = int(tmp[1])
        return n_test, m_test

    # Store all the test cases
    test_cases = []
    for i in range(n):
        n_test, m_test = read_case(i)
        test_cases.append((n_test, m_test))

    print("Total number of test cases:", len(test_cases))
    
    return test_cases

In [22]:
test_cases_s = read_data_file(filename_s)

n = 100
Total number of test cases: 100


### Solution

The number of ways is the same as the total number of ways of sitting for N couples minus the total number of ways that there is at least one newlywed couple sitting together. 

To calculate the total number of ways that there is at least one newlywed couple sitting together, we can use "Union of events" in probability theory.

Reference:
https://www.thoughtco.com/probability-union-of-three-sets-more-3126263

1. Add the ways of each newlywed couple sitting together.
2. Subtract the ways of the intersections of every pair of newlywed couples sitting together.
3. Add the ways of the intersection of every set of three newlywed couples sitting together.
4. Subtract the ways of the intersection of every set of four newlywed couples sitting together.
5. Continue this process until the last is the ways of the intersection of the total number of newlywed couples that we started with.

In [23]:
import math
from scipy.special import comb
from tqdm import *

def solve_small(test_cases):
    """Return the result to small test cases"""
    
    # Each case
    result = []
    for n_test, m_test in tqdm(test_cases):
        #print(n_test, m_test)
        
        # Total number of ways for N couples
        count = math.factorial(2*n_test) 
        
        # Calculate the union of events for M newlywed couples sitting together
        temp_count = []
        temp_count.append(math.factorial(2 * (n_test - 1) + 1) * 2 * m_test)
        for i in range(m_test - 1):
            x = i + 2
            temp_count.append(((-1) ** (x - 1)) * comb(m_test, x, exact=True) * math.factorial(2 * (n_test - x) + x) * (2 ** x))
        
        #print(temp_count)
        # Total number of ways for N couples - ways for M newlywed couples sitting together
        result.append((count - sum(temp_count)) % (10 ** 9 + 7))

    print("Done")
    return result

In [24]:
result_s = solve_small(test_cases_s)

100%|██████████| 100/100 [00:00<00:00, 3701.68it/s]


Done


### Prepare output

In [25]:
def write_output_file(result, filename):
    """Write result to an output file"""
    
    with open(filename, "w") as f:
        for i, r in enumerate(result):
            f.write("Case #" + str(i + 1) + ": " + str(r) + "\n")

    print("Done")

In [26]:
write_output_file(result_s, "C-small-practice.out")

Done


### Prepare Test Cases - Solve A-large

In [27]:
filename_l = "C-large-practice.in"

In [28]:
test_cases_l= read_data_file(filename_l)

n = 100
Total number of test cases: 100


### Solution

For the large dataset, we need to speed up by:

1. Precompute modulos of factorials, modulo inverses of factorials and powers of 2
2. Use modulos of factorials, modulo inverses of factorials to compute nCr
3. Apply modular addition, subtraction and muliplication

Reference: 
- https://www.quora.com/How-do-I-find-the-value-of-nCr-1000000007-for-the-large-number-n-n-10-6-in-C
- https://www.khanacademy.org/computing/computer-science/cryptography#modarithmetic

In [29]:
import math
import gmpy2
from tqdm import *

# Precompute modulos of factorials
pre_factorial = {}
pre_factorial[0] = 1

for i in tqdm(range(1, 200001)):
    pre_factorial[i] = (pre_factorial[i - 1] * i) % (10 ** 9 + 7)

# Precompute modulo inverses of factorials
pre_ifactorial = {}
pre_ifactorial[0] = 1

for i in tqdm(range(1, 200001)):
    pre_ifactorial[i] = pre_ifactorial[i - 1] * int(gmpy2.invert(i, 10 ** 9 + 7)) % (10 ** 9 + 7)

# Precompute powers of 2
two_powers = {}

two_tmp = 1
for i in tqdm(range(1, 100001)):
    two_tmp *= 2
    two_powers[i] = two_tmp % (10 ** 9 + 7)


100%|██████████| 100000/100000 [00:01<00:00, 51481.55it/s]


In [30]:
import math
from scipy.special import comb
from tqdm import *

def solve_large(test_cases):
    """Return the result to large test cases"""
    
    # Each case
    result = []
    for n_test, m_test in tqdm(test_cases):
        #print(n_test, m_test)
        
        # Modular addition and subtraction
        # (A + B) mod C = (A mod C + B mod C) mod C
        count = math.factorial(2*n_test) % (10 ** 9 + 7)
        
        temp_count = []
        for x in range(m_test, 0, -1):
            #print(x)
            # Calculate nCr mod (10 ** 9 + 7) using modulos of factorials and modulo inverses of factorials
            c = pre_factorial[m_test] * pre_ifactorial[x] * pre_ifactorial[m_test - x] % (10 ** 9 + 7)
            
            # Modular multiplication
            # (A * B) mod C = (A mod C * B mod C) mod C
            temp_count.append(((-1) ** (x - 1)) * c * pre_factorial[2 * (n_test - x) + x] * two_powers[x] % (10 ** 9 + 7))
        
        #print(temp_count)
        result.append((count - sum(temp_count)) % (10 ** 9 + 7))

    print("Done")
    return result

In [32]:
result_l = solve_large(test_cases_l)

100%|██████████| 100/100 [00:54<00:00,  1.83it/s]


Done


### Prepare output

In [33]:
write_output_file(result_l, "C-large-practice.out")

Done
